In [1]:
# define categories and corresponding urls to be scraped

def input_category_information(name, pages, recipe_number):
    # input information
    dic = {}
    dic['name'] = name
    dic['pages'] = pages
    dic['number'] = recipe_number

    return dic

def define_required_data():
    
    raw_data = []
    #aw_data.append(input_category_information('burger', 18, 421))
    #raw_data.append(input_category_information('salad', 53, 1271))
    raw_data.append(input_category_information('cake', 36, 857))
    #raw_data.append(input_category_information('burger', 1, 421))
    #raw_data.append(input_category_information('salad', 1, 421))
    return raw_data


def get_urls(dic):
    # get urls for each category
    urls = []
    for i in range(1, dic['pages']+1):
        url = 'https://www.bbcgoodfood.com/search/recipes/page/' + str(i) + '/?q=' + dic['name'] + '&sort=-relevance'
        urls.append(url)
    
    return urls


In [2]:
# scrape information from defined urls

import requests
import os
import pandas as pd
from recipe_scrapers import scrape_me


def scrape_from_web(categories):
    for category in categories:
        name = category['name']
        recipe_links = []
        urls = get_urls(category)
        
        if not os.path.exists('./extracted_data/'):
            os.mkdir('./extracted_data/')
        
        path = './extracted_data/' +  name + '/'
        if not os.path.exists(path):
            os.mkdir(path)
        
        
        # loop through pages
        for url in urls:
            # scrape from table of contents
            scraper = scrape_me(url)
            # extract all the links in this page
            links = scraper.links()
            # extract recipe links in this page and append in the recipe links
            recipe_links = extract_recipe_links(recipe_links, links)
        
        # store recipe links?
        #recipe_links.to_csv(path + name + '_recipe_links.csv', index=False)
        
        # extract information from each recipe
        structured_information = extract_from_recipes(recipe_links)
        
        # download images from extracted image links
        download_images(path, structured_information)

        df = pd.DataFrame(structured_information)
        df.to_csv('./csv_files/' + name + '.csv', index=False)

        return structured_information

        
def extract_recipe_links(recipe_links, links):
    # extract recipe links from all the links

    for link in links:
        try:
            if link['class'] == ['standard-card-new__article-title', 'qa-card-link']:
                recipe_links.append('https://www.bbcgoodfood.com'+link['href'])
        except:
            continue
    
    return recipe_links


def extract_from_recipes(recipe_links):
    # combine structured information of each recipe
    structured_information = []
    i = 1
    for recipe_link in recipe_links:
        print(i)
        i+=1
        temp = structured_information_in_recipe(recipe_link)
        # remove recipes that contain incomplete information
        if temp == {} or temp['nutrients'] == {} or temp['ingredients'] == {} or temp['image'] == {}:
            continue
        else:
            structured_information.append(temp)
    return structured_information


def structured_information_in_recipe(recipe_link):
    # extract name, image, ingredients, and nutrients from each recipe website
    recipe = {}
    scraper = scrape_me(recipe_link)
    recipe['title'] = scraper.title()
    recipe['image'] = scraper.image()
    recipe['ingredients'] = scraper.ingredients()
    recipe['nutrients'] = scraper.nutrients()
    
    return recipe

    
def download_images(path, structured_information):
    # download images from image links
    index = 0
    
    #path = './extracted_data/' +  category_name + '/'
    #if not os.path.exists(path):
    #    os.mkdir(path)
    
    for recipe in structured_information:
        # if image is png then save as .png
        
        image = requests.get(recipe['image'])
        with open(path + str(index)+'.jpg', 'wb') as f:
            f.write(image.content)
        index += 1


In [3]:
import sys
sys.setrecursionlimit(20000)

#define categories to be extracted
categories = define_required_data()
structured_information = scrape_from_web(categories)




1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [4]:
structured_information

[{'title': 'Easy chocolate fudge cake',
  'image': 'https://images.immediate.co.uk/production/volatile/sites/30/2020/08/chocolate-fudge-cake-91de17a.jpg',
  'ingredients': ['150ml sunflower oil, plus extra for the tin',
   '175g self-raising flour',
   '2 tbsp cocoa powder',
   '1 tsp bicarbonate of soda',
   '150g caster sugar',
   '2 tbsp golden syrup',
   '2 large eggs, lightly beaten',
   '150ml semi-skimmed milk',
   '100g unsalted butter',
   '225g icing sugar',
   '40g cocoa powder',
   '2½ tbsp milk (a little more if needed)'],
  'nutrients': {'calories': '608 calories',
   'fatContent': '33 grams fat',
   'saturatedFatContent': '11 grams saturated fat',
   'carbohydrateContent': '69 grams carbohydrates',
   'sugarContent': '52 grams sugar',
   'fiberContent': '2 grams fiber',
   'proteinContent': '7 grams protein',
   'sodiumContent': '0.7 milligram of sodium'}},
 {'title': 'Chocolate marble cake',
  'image': 'https://images.immediate.co.uk/production/volatile/sites/30/2020/08

# Split into train and test sets (8:2)

In [44]:
import pandas as pd
import os
import shutil

categories = ['burger', 'salad', 'cake']
#categories = ['burger']


for category in categories:
    path = './csv_files/'
    for path, subpath, files in os.walk(path):
        files.sort()
        for i in files:
            if i.endswith(category + ".csv"):
                data = pd.read_csv(path + i)
                train_data = data[:-round(len(data)*0.2)]
                test_data = data[-round(len(data)*0.2):]
                train_data.to_csv('./extracted_data/' + category + '/train_' + category + '.csv', index=False)
                test_data.to_csv('./testset/' + category + '/test_' + category + '.csv', index=False)
                for i in range(len(test_data)):
                    shutil.move('./extracted_data/' + category + '/' + str(len(data)-i-1) + '.jpg',
                                './testset/' + category + '/')


# Integrate all the train and test data

In [43]:
import pandas as pd
import os
import shutil

categories = ['burger', 'salad', 'cake']

for category in categories:
    src = './testset/' + category + '/'
    dst = './extracted_data/' + category + '/'
    
    try:
        os.remove(src + 'test_' + category + '.csv')
        os.remove(dst + 'train_' + category + '.csv')
        for path, subpath, files in os.walk(src):
            files.sort()
            for i in files:
                if i.endswith(".jpg"):
                    shutil.move(src + i, dst + i)   
    except:
        for path, subpath, files in os.walk(src):
            files.sort()
            for i in files:
                if i.endswith(".jpg"):
                    shutil.move(src + i, dst + i)   




In [1]:
import requests
import os
import pandas as pd
from recipe_scrapers import scrape_me


scraper = scrape_me('https://www.bbcgoodfood.com/recipes/summer-courgette-ribbon-salad')
scraper.image()
# 这个爬不出来正确的网址
# https://images.immediate.co.uk/production/volatile/sites/30/2020/08/courgette-ribbon-salad-c2afab4.png?quality=90&resize=600,545

'https://images.immediate.co.uk/production/volatile/sites/30/2020/08/courgette-ribbon-salad-c2afab4.png'